# <font size="4" color='black'>Self Implementation of DefaultModelMonitor</font>

Define a simple regression use case

This use case has two columns. Age and Salary. Our input is Age and label is Salary. 
We are looking to build models and predict Salary for any given Age.
The purpose is to demonstrate how this can be done using MLOps.

## <center><font size="5" color='black'> ------------- Section 01 - FE and Training ------------- </font>

## <font size="3" color='blue'>Get Input DataSet, perform FE, train-test split</font>

In [ ]:
# get London time

from datetime import datetime
import pytz
import os

def read(s3_url):
    
    file_path = fr'{s3_url}'
    file_name = os.path.basename(file_path)
    
    !rm ./tmp/$file_name 2> /dev/null
    
    # print verbose if error
    !aws s3 cp $file_path  ./tmp/ | grep 'fatal error'
    !cat ./tmp/$file_name
    
def get_london_time():
    tz_London = pytz.timezone('Europe/London')
    tm_London = datetime.now(tz_London).strftime("%Y%m%d%H%M%S")
    return tm_London

In [ ]:
read("s3://madan-s3/pipelines/use_case_ageVsSalary/batchoutput/test_df.csv.out")

### <font size="2" color='blue'>Input DataSet<font>

In [ ]:
#Sandeep
#bucket_name= "01-attempts-406016308324"

#Madan
bucket_name= "madan-s3"
root_s3_folder = fr's3://{bucket_name}/pipelines/use_case_ageVsSalary'

print(root_s3_folder)

In [ ]:
# A SageMaker session is needed to establish a connection between the notebook and AWS SageMaker services, enabling streamlined management of resources like training jobs and model deployments. It simplifies interactions with SageMaker by abstracting complex API calls, facilitating efficient machine learning workflows.

import sagemaker
from sagemaker.session import Session

sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

In [ ]:
import pandas as pd

In [ ]:
input_data_for_training = pd.read_csv(r'../data/input-data-for-training.csv')
input_data_for_training.head()

In [ ]:
import matplotlib.pyplot as plt

x = input_data_for_training.age
y = input_data_for_training.salary

plt.scatter(x, y)
plt.title("Age-Vs-Salary Distribution")
plt.xlabel("Age")
plt.ylabel("Salary")
plt.show()

In [ ]:
# Split the dataset for training

from sklearn.model_selection import train_test_split 


X_train, X_test, y_train, y_test = train_test_split(
                                    input_data_for_training.age, 
                                    input_data_for_training.salary,
                                    random_state=104,  
                                    test_size=0.3,  
                                    shuffle=True
                                    ) 

In [ ]:
print(f'input_data_for_training.shape : {input_data_for_training.shape}')
print(f'X_train : {len(X_train)} y_train : {len(y_train)}')
print(f'X_test  : {len(X_test)} y_test  : {len(y_test)}')

### <font size="2" color='blue'>Create a dummy feature engineering function<font>

In [ ]:
def perform_feature_engineering(raw_data_df):
    # this is a dummy feature engineering function.
    # in real-case, it can be replace with more sophisticated code
    # that perform feature engineering and derive new features
    return raw_data_df

In [ ]:
input_data_for_training = perform_feature_engineering(input_data_for_training)
input_data_for_training.head(5)

In [ ]:
train_df = pd.DataFrame({'y_salary' : y_train, 'x_age' : X_train})
print(f'train_df.shape : {train_df.shape}')
train_df.head(3)

In [ ]:
validation_df = pd.DataFrame({'y_salary' : y_test, 'x_age' : X_test})
print(f'validation_df.shape : {validation_df.shape}')
validation_df.head(3)

In [ ]:
train_df.to_csv(r'../data/train_df.csv', index=False, header=False)
validation_df.to_csv(r'../data/validation_df.csv', index=False, header=False)

In [ ]:
#Madan: create function to save files into S3
import boto3

def upload_file(file_name, bucket, object_name=None):
    if object_name is None:
        object_name = file_name
        
    # Upload the file
    s3 = boto3.client('s3')
    try:
        s3.upload_file(file_name, bucket, object_name)
        print(f"Uploaded {file_name} to S3 bucket {bucket} as {object_name}")
    except Exception as e:
        print(f"Error uploading file: {e}")

In [ ]:
#Madan:  upload Train into s3
file_name = r'../data/train_df.csv'
object_name= "pipelines/use_case_ageVsSalary/input_data_for_training/train/train_df.csv"
upload_file(file_name,bucket_name,object_name)


file_name = r'../data/validation_df.csv'
object_name= "pipelines/use_case_ageVsSalary/input_data_for_training/valid/validation_df.csv"
upload_file(file_name,bucket_name,object_name)

### <font size="2" color='blue'>Retrieve base image URI and run the training through processing job<font>

In [ ]:
# getting the linear learner image according to the region
import boto3
from sagemaker.image_uris import retrieve

base_image = retrieve("linear-learner", boto3.Session().region_name, version="1")
print(f'base_image : {base_image}')

In [ ]:
train_location=fr'{root_s3_folder}/input_data_for_training/train/train_df.csv'
validation_location=fr'{root_s3_folder}/input_data_for_training/valid/validation_df.csv'

train_config=sagemaker.session.TrainingInput(
    s3_data=train_location,
    content_type='text/csv',
    s3_data_type='S3Prefix')

validation_config=sagemaker.session.TrainingInput(
    s3_data=validation_location,
    content_type='text/csv',
    s3_data_type='S3Prefix')

data_channels={'train':train_config,'validation':validation_config}

print(train_config.config)
print(validation_config.config)

In [ ]:
output_location = fr'{root_s3_folder}/output/'

linear_estimator = sagemaker.estimator.Estimator(
                                                    base_image,
                                                    role,
                                                    input_mode="File",
                                                    instance_count=1,
                                                    instance_type="ml.m4.xlarge",
                                                    output_path=output_location,
                                                    sagemaker_session=sagemaker_session
                                                )

In [ ]:
# Note: set hyperparameter as training job was failing with below error </br></br>

# ClientError: No value(s) were specified for 'predictor_type' which are required hyperparameter(s) (caused by ValidationError), exit code: 2
# set hyperparameters 
linear_estimator.set_hyperparameters(
        feature_dim=1,
        mini_batch_size=1,
        predictor_type='regressor')

In [ ]:
# this will trigger processing job

# Concise description of each SageMaker job:

# Training Job: Trains a machine learning model using a specified algorithm and dataset.
# Processing Job: Executes data preprocessing, postprocessing, or any custom processing tasks.
# Transform Job (Batch Inference Job): Applies a trained model to a large dataset for batch predictions.
# Inference Job (Endpoint): Deploys a trained model as a real-time endpoint for online predictions.
# Hyperparameter Tuning Job: Automatically searches for the best hyperparameters to optimize model performance.
# Data Wrangler Job: Processes and visualizes data using a graphical interface to prepare it for machine learning tasks.

# defining train_data, the LinearLearner algorithm expects the data to be in a specific format, typically as a CSV file or in the record-IO protobuf format. The dataset should be in a tabular format where the first column is the label (target variable) and the remaining columns are the features.


job_name = "AgeVsSalary-linear-learner-train-job-" + get_london_time()
print("Training job", job_name)

linear_estimator.fit(inputs=data_channels, job_name=job_name)

### <font size="2" color='blue'>Create simple function to print evaluation metrics</font>

### <font size="2" color='blue'>Store the trained model image for repeated use</font>

In [ ]:
trained_model=fr'{output_location}{job_name}/output/model.tar.gz'
print(trained_model)

In [ ]:
# remember to delete endpoint after use

model_name="AgeVsSalary-model-" + get_london_time()
endpoint_name = "AgeVsSalary-Endpoint-" + get_london_time()

# predictor = linear_estimator.deploy(
#     initial_instance_count=1,
#     instance_type='ml.m4.xlarge',
#     endpoint_name = endpoint_name)

In [ ]:
# from sagemaker.serializers import CSVSerializer

# predictor.serializer = CSVSerializer()
# result = predictor.predict([[30]])
# print(result)

### <font size="2" color='blue'>Delete model to avoid unnecessary cost</font>

In [ ]:
# predictor.delete_endpoint()

### <font size="2" color='blue'>Retriev stored model and perform new prediction</font>

In [ ]:
from sagemaker.model import Model
from sagemaker.predictor import Predictor

#stored_model_artifact = fr'{root_s3_folder}/output/AgeVsSalary-linear-learner-train-job-20240809153258/output/model.tar.gz'
stored_model_artifact= trained_model
stored_model = Model(
                        image_uri=base_image,
                        model_data=stored_model_artifact,
                        role=role,
                        name=model_name,
                        predictor_cls=Predictor
                        )

In [ ]:
# remember to delete endpoint
print(fr'endpoint_name : {endpoint_name}')

predictor = stored_model.deploy(
                                    initial_instance_count=1,
                                    instance_type='ml.m4.xlarge',
                                    endpoint_name = endpoint_name
                                )

In [ ]:
print(type(predictor))

In [ ]:
from sagemaker.serializers import CSVSerializer

predictor.serializer = CSVSerializer()
result = predictor.predict([[30]])
print(result)

In [ ]:
predictor.delete_endpoint()

## <center><font size="5" color='black'> ------------- Section 02 - Batch Transform ------------- </font>

## <font size="3" color='green'>Create Code to input data, generate inferences using batch transform</font>

### <font size="2" color='green'>Get a very simple dataset of age and salary with some randomness</font> 

1. See data folder. 

### <font size="2" color='green'>Create a code to generate inference using batch transform</font> 

### <font size="2" color='green'>Provide a sample batch input file and generate inferences</font>

### <font size="2" color='green'>Create sagemaker pipeline to train model and generate inference again when a new train input file is loaded</font>

## <center><font size="5" color='black'> ------------- Section 03 - Pipeline ------------- </font>

In [1]:
#Sandeep
#bucket_name= "01-attempts-406016308324"

#Madan
bucket_name= "madan-s3"
root_s3_folder = fr's3://{bucket_name}/pipelines/use_case_ageVsSalary'

print(root_s3_folder)

s3://madan-s3/pipelines/use_case_ageVsSalary


In [2]:

import sagemaker
from sagemaker.session import Session

sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


## <font size="3" color='red'>1. Create a pipeline to train data, create model and register model</font>

## <font size="3"> Preprocessing step </font>

In [3]:
%%writefile preprocessing.py

from datetime import datetime
import pytz
import os
import pandas as pd
#import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split 


def perform_feature_engineering(raw_data_df):
    # this is a dummy feature engineering function.
    # in real-case, it can be replace with more sophisticated code
    # that perform feature engineering and derive new features
    return raw_data_df

if __name__ == "__main__":
    
 base_dir = "/opt/ml/processing"
input_data_for_training = pd.read_csv(fr'{base_dir}/input/input-data-for-training.csv')
#input_data_for_training.head()

# x = input_data_for_training.age
# y = input_data_for_training.salary

# plt.scatter(x, y)
# plt.title("Age-Vs-Salary Distribution")
# plt.xlabel("Age")
# plt.ylabel("Salary")
# plt.show()

# Split the dataset for training
X_train, X_test, y_train, y_test = train_test_split(
                                    input_data_for_training.age, 
                                    input_data_for_training.salary,
                                    random_state=104,  
                                    test_size=0.3,  
                                    shuffle=True
                                    ) 

print(f'input_data_for_training.shape : {input_data_for_training.shape}')
print(f'X_train : {len(X_train)} y_train : {len(y_train)}')
print(f'X_test  : {len(X_test)} y_test  : {len(y_test)}')

input_data_for_training = perform_feature_engineering(input_data_for_training)
#input_data_for_training.head(5)

train_df = pd.DataFrame({'y_salary' : y_train, 'x_age' : X_train})
print(f'train_df.shape : {train_df.shape}')
#train_df.head(3)

validation_df = pd.DataFrame({'y_salary' : y_test, 'x_age' : X_test})
print(f'validation_df.shape : {validation_df.shape}')
#validation_df.head(3)

train_df.to_csv(fr'{base_dir}/train/train_df.csv', index=False, header=False)
validation_df.to_csv(fr'{base_dir}/validation/validation_df.csv', index=False, header=False)

Overwriting preprocessing.py


In [4]:
import boto3

def upload_file(file_name, bucket, object_name=None):
    if object_name is None:
        object_name = file_name
        
    # Upload the file
    s3 = boto3.client('s3')
    try:
        s3.upload_file(file_name, bucket, object_name)
        print(f"Uploaded {file_name} to S3 bucket {bucket} as {object_name}")
    except Exception as e:
        print(f"Error uploading file: {e}")

In [5]:
#upload source file to S3 as this'll get use during preprocessing step in pipeline

file_name = r'../data/input-data-for-training.csv'
object_name= "pipelines/use_case_ageVsSalary/input_data_for_training/sourcefile/input-data-for-training.csv"
upload_file(file_name,bucket_name,object_name)


Uploaded ../data/input-data-for-training.csv to S3 bucket madan-s3 as pipelines/use_case_ageVsSalary/input_data_for_training/sourcefile/input-data-for-training.csv


In [6]:
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.workflow.pipeline_context import PipelineSession
pipeline_session = PipelineSession()

sklearn_processor = SKLearnProcessor(
    framework_version="1.2-1",
    instance_type="ml.m5.xlarge",
    instance_count=1,
    base_job_name="AgeVsSalary-process",
    role=role,
    sagemaker_session=pipeline_session,
)

In [7]:
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.workflow.steps import ProcessingStep

source_input_data=f'{root_s3_folder}/input_data_for_training/sourcefile/input-data-for-training.csv'

processor_args = sklearn_processor.run(
    inputs=[
        ProcessingInput(source=source_input_data, destination="/opt/ml/processing/input"),
    ],
    outputs=[
        ProcessingOutput(output_name="train", source="/opt/ml/processing/train"),
        ProcessingOutput(output_name="validation", source="/opt/ml/processing/validation"),
        ProcessingOutput(output_name="test", source="/opt/ml/processing/test")
    ],
    code="preprocessing.py",
)

step_process = ProcessingStep(name="AgeVsSalaryProcess", step_args=processor_args)

/opt/conda/lib/python3.10/site-packages/sagemaker/workflow/pipeline_context.py:332: UserWarning: Running within a PipelineSession, there will be No Wait, No Logs, and No Job being started.
  warnings.warn(


In [8]:
from datetime import datetime
import pytz

def get_london_time():
    tz_London = pytz.timezone('Europe/London')
    tm_London = datetime.now(tz_London).strftime("%Y%m%d%H%M%S")
    return tm_London

## <font size="3"> training step </font>

In [9]:
# create training step process

import boto3
from sagemaker.image_uris import retrieve
from sagemaker.inputs import TrainingInput
from sagemaker.workflow.steps import TrainingStep


job_name = "AgeVsSalary-train-pipeline-" + get_london_time()
print("Training job", job_name)
output_location = fr'{root_s3_folder}/output/'


base_image = retrieve("linear-learner", boto3.Session().region_name, version="1")

linear_estimator = sagemaker.estimator.Estimator(
                                                    base_image,
                                                    role,
                                                    input_mode="File",
                                                    instance_count=1,
                                                    instance_type="ml.m4.xlarge",
                                                    output_path=output_location,
                                                    sagemaker_session=pipeline_session
                                                )

linear_estimator.set_hyperparameters(
        feature_dim=1,
        mini_batch_size=1,
        predictor_type='regressor')

train_config=sagemaker.session.TrainingInput(
    s3_data=step_process.properties.ProcessingOutputConfig.Outputs[
                "train"
            ].S3Output.S3Uri,
    content_type='text/csv',
    s3_data_type='S3Prefix')

validation_config=sagemaker.session.TrainingInput(
    s3_data=step_process.properties.ProcessingOutputConfig.Outputs[
                "validation"
            ].S3Output.S3Uri,
    content_type='text/csv',
    s3_data_type='S3Prefix')

data_channels={'train':train_config,'validation':validation_config}


train_fit= linear_estimator.fit(inputs=data_channels, job_name=job_name)

step_train=  TrainingStep(
    name= "AgeVsSalary-train",
    step_args=train_fit)

Training job AgeVsSalary-train-pipeline-20240821041941


## <font size="3"> Model evalution script </font>

In [10]:
%%writefile evaluate.py

import json
import logging
import os
import pickle
import tarfile

import pandas as pd
from sklearn.metrics import mean_squared_error, r2_score

logger = logging.getLogger()
logger.setLevel(logging.INFO)
logger.addHandler(logging.StreamHandler())


if __name__ == "__main__":
    model_path = "/opt/ml/processing/model/model.tar.gz"
    with tarfile.open(model_path) as tar:
        tar.extractall(path="..")

    logger.debug("Loading linear regression model.")
    model = pickle.load(open("linear_regression_model", "rb"))

    print("Loading test input data")
    test_path = "/opt/ml/processing/test/validation_df.csv"
    df = pd.read_csv(test_path, header=None)

    logger.debug("Reading test data.")
    y_test = df.iloc[:, 0].to_numpy()
    df.drop(df.columns[0], axis=1, inplace=True)
    X_test = df.values

    logger.info("Performing predictions against test data.")
    predictions = model.predict(X_test)

    logger.info("Evaluating model performance.")
    mse = mean_squared_error(y_test, predictions)
    r2 = r2_score(y_test, predictions)

    # Update report dictionary with chosen metrics
    report_dict = {
        "regression_metrics": {
            "mean_squared_error": {
                "value": mse,
                "standard_deviation": "NaN"
            },
            "r2_score": {
                "value": r2,
                "standard_deviation": "NaN"
            }
        }
    }

    print("Regression evaluation report:\n{}".format(report_dict))

    evaluation_output_path = os.path.join("/opt/ml/processing/evaluation", "evaluation.json")
    print("Saving evaluation report to {}".format(evaluation_output_path))

    with open(evaluation_output_path, "w") as f:
        f.write(json.dumps(report_dict))

Overwriting evaluate.py


In [11]:
from sagemaker.processing import ScriptProcessor


script_eval = ScriptProcessor(
    image_uri= base_image,
    command=["python3"],
    instance_type="ml.m5.xlarge",
    instance_count=1,
    base_job_name="AgeVsSalary-evalution",
    role=role,
    sagemaker_session=pipeline_session,
)

eval_args = script_eval.run(
    inputs=[
        ProcessingInput(
            source=step_train.properties.ModelArtifacts.S3ModelArtifacts,
            destination="/opt/ml/processing/model",
        ),
        ProcessingInput(
            source=step_process.properties.ProcessingOutputConfig.Outputs["test"].S3Output.S3Uri,
            destination="/opt/ml/processing/test",
        ),
    ],
    outputs=[
        ProcessingOutput(output_name="evaluation", source="/opt/ml/processing/evaluation"),
    ],
    code="evaluate.py",
)

In [12]:
from sagemaker.workflow.properties import PropertyFile


evaluation_report = PropertyFile(
    name="EvaluationReport", output_name="evaluation", path="evaluation.json"
)
step_eval = ProcessingStep(
    name="AgevsSalEval",
    step_args=eval_args,
    property_files=[evaluation_report],
)

In [13]:
from sagemaker.model_metrics import MetricsSource, ModelMetrics
model_metrics = ModelMetrics(
    model_statistics=MetricsSource(
        s3_uri="{}/evaluation.json".format(
            step_eval.arguments["ProcessingOutputConfig"]["Outputs"][0]["S3Output"]["S3Uri"]
        ),
        content_type="application/json",
    )
)

## <font size="3"> Model create and registor </font>

In [14]:
# Create model and register
from sagemaker.model import Model
from sagemaker.inputs import CreateModelInput
from sagemaker.workflow.model_step import ModelStep


model = Model(
    image_uri=base_image,
    model_data=step_train.properties.ModelArtifacts.S3ModelArtifacts,
    sagemaker_session=pipeline_session,
    role=role,
)

step_create_model = ModelStep(
    name="AgeVsSalaryModel",
    step_args=model.create(instance_type="ml.m5.large", accelerator_type="ml.eia1.medium"),
)

register_args = model.register(
    content_types=["text/csv"],
    response_types=["text/csv"],
    inference_instances=["ml.t2.medium", "ml.m5.xlarge"],
    transform_instances=["ml.m5.xlarge"],
    model_package_group_name="AgeVsSalaryGroup",
    approval_status="PendingManualApproval",
    model_metrics=model_metrics,
   
)
step_register = ModelStep(name="AgeVsSalaryModelStep", step_args=register_args)

## <font size="3"> Pipeline for preprocess,train and model </font>

In [15]:
## pipeline

from sagemaker.workflow.pipeline import Pipeline


pipeline_name = "AgeVsSalary-Pipeline"
pipeline = Pipeline(
    name=pipeline_name,
    parameters=[
        1,
        "ml.m5.xlarge"
    ],
    steps=[step_process,step_train, step_create_model,step_register],
   )

In [16]:
#run pipeline
pipeline.upsert(role_arn=role)

{'PipelineArn': 'arn:aws:sagemaker:us-east-1:908263978175:pipeline/AgeVsSalary-Pipeline',
 'ResponseMetadata': {'RequestId': '1ac3aeca-406f-4ca7-b791-e2b9b3db4fde',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '1ac3aeca-406f-4ca7-b791-e2b9b3db4fde',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '88',
   'date': 'Wed, 21 Aug 2024 03:20:49 GMT'},
  'RetryAttempts': 0}}

In [17]:
execution= pipeline.start()

## <font size="3"> pipeline for batch transform when model approve </font>

In [ ]:
import pandas as pd
test_age=[30,20,12]
test_df=pd.DataFrame(test_age)
test_df.to_csv(r'../data/test_df.csv', index=False, header=False)

In [ ]:
file_name = r'../data/test_df.csv'
object_name= "pipelines/use_case_ageVsSalary/input_data_for_training/test/test_df.csv"
upload_file(file_name,bucket_name,object_name)

In [ ]:
from sagemaker.transformer import Transformer

test_s3 = f'{root_s3_folder}/input_data_for_training/test/test_df.csv'
batch_output = f'{root_s3_folder}/batchoutput'

transformer = Transformer(
    model_name="pipelines-q7ccj9vacdej-AgeVsSalaryModel-Cre-zSoG4h7s4A", #step_create_model.properties.ModelName,
    instance_type="ml.m5.xlarge",
    instance_count=1,
    output_path=batch_output
   
)

from sagemaker.inputs import TransformInput
from sagemaker.workflow.steps import TransformStep


step_transform = TransformStep(
    name="AgeVsSalaryTransform", transformer=transformer, inputs=TransformInput(data=test_s3,content_type="text/csv",split_type="Line")
)

#pipeline
from sagemaker.workflow.pipeline import Pipeline


pipeline_name = "AgeVsSalary-batch-pipeline"
pipeline = Pipeline(
    name=pipeline_name,
    parameters=[
        1,
        "ml.m5.xlarge"
    ],
    steps=[step_transform],
   )

#run pipeline
pipeline.upsert(role_arn=role)
execution= pipeline.start()

In [ ]:
print(batch_output)

## <font size="3" color="red"> pending -> </br> 1. get model name which pending for approval </br> 2. create event bridge </br> a. set rule- > when model approved </br> b. target -> trigger pipeline which will run transform job
</font>

## <font size="3"> Create event bridge when model approve </font>

In [ ]:
import boto3
import json
import sagemaker
from sagemaker import get_execution_role

#role= "arn:aws:iam::908263978175:role/lambda-sagemaker-pipeline"

eb_client = boto3.client('events')
rule_name="event-agevssal-transform-job-sdk"

#set rule , logic referred with  console working event
response = eb_client.put_rule(
        Name=rule_name,
        EventPattern=json.dumps({
          "source": ["aws.sagemaker"],
          "detail-type": ["SageMaker Model Package State Change"],
          "detail": {
            "ModelPackageGroupName": ["AgeVsSalaryGroup"],
            "ModelApprovalStatus": ["Approved"]
          }
        }))

session = boto3.session.Session()
region = session.region_name
account_id = boto3.client('sts').get_caller_identity()['Account']
target_arn = f"arn:aws:lambda:{region}:{account_id}:function:run-agevssal-pipeline"

print(target_arn)

eb_client.put_targets(
    Rule=rule_name,
    Targets=[{
        "Id": "target-agevssal-transform-job-sdk",
        "Arn": target_arn
    }]
    )


## <font size="3" color='red'>Create a sample pipeline on existing train data</font>

### <font size="2" color='red'>Execute pipeline and generate inference on existing train data</font>

### <font size="2" color='red'>Upload new train data and generate inference by executing pipeline</font>

## <center><font size="5" color='black'> ------------- Section 04 - DefaultModelMonitor ------------- </font>

## <font size="3" color='blue'>DefaultModelMonitor : Create code to monitor data quality and accept/reject batch input file based on constraints</font>

### <font size="2" color='blue'>Code for monitor data quality config and job using DefaultModelMonitor</font>

### <font size="2" color='blue'>Get a batch input file with outlier that the monitor data quality step should REJECT</font>

### <font size="2" color='blue'>Get a batch input file with outlier that the monitor data quality step should ACCEPT</font>

## <center><font size="5" color='black'> ------------- Section 05 - ModelMonitor ------------- </font>

## <font size="3" color='green'>ModelMonitor : Create code to monitor data quality and accept/reject batch input file based on constraints</font>

### <font size="2" color='green'>Code for monitor data quality config and job using ModelMonitor</font>

### <font size="2" color='green'>Get a batch input file with outlier that the monitor data quality step should REJECT</font>

### <font size="2" color='green'>Get a batch input file with outlier that the monitor data quality step should ACCEPT</font>

# <font size="4" color='black'>End Of Notebook</font>